<a href="https://colab.research.google.com/github/Fragoa/Image-Coloring-with-AE-and-VAE/blob/main/ImageColoring_AE_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>